In [ ]:
import socket
import time
import numpy as np
import torch
from Res import *
from MCTS import *
from Connect2Game import Connect2Game
from go import *

In [ ]:
# Escolha do board para o jogo
Games = ["G7x7", "G9x9"]
number = int(input("Escolha o jogo: 1-7x7 2-9x9: "))
Ga = Games[number-1]

# Função para obter o tamanho do board
def n_board(Game):
    n = int(Game[1])
    return n

# Configuração do board
n_board = n_board(Ga)

# Inicia o game para usar no mcts
game = Connect2Game(n_board)

# Escolha do device 
device = torch.device("cpu")

# Argumentos para o mcts
args = {
    'C': 2,
    'num_searches': 1,
    'num_iterations': 1,
    'num_selfPlay_iterations': 1,
    'num_parallel_games': 1,
    'num_epochs': 1,
    'batch_size': 128,
    'temperature': 1.25,
    
    'dirichlet_epsilon': 0.25,
    'dirichlet_alpha': 0.3
}

# Inicia o modelo
model = ResNet(game, 9, 128, device)

# Load ao modelo já treinado
model.load_state_dict(torch.load("model_0_Go.pt", map_location=torch.device('cpu')))
model.eval()

# Inicia o mcts
mcts = MCTS(game, args, model)

# Função para gerar o move do agente
def generate_move_from_model(state, previous_state, ag):
    #Verifica se é o agente 1 ou 2
    if ag ==1:
        player = 1
    else:
        player = -1
    #Muda a perspectiva do tabuleiro para usar no mcts
    neutral_state = game.change_perspective(state, player)
    neutral_state_previous = game.change_perspective(previous_state, player)
    
    # Usa o mcts para obter as probabilidades
    mtcs_probs = mcts.search(neutral_state, neutral_state_previous)
    
    # Escolhe a ação com maior probabilidade
    action = np.argmax(mtcs_probs)
    
    # Se a ação for a ultima da lista, retorna pass
    if action == game.action_size - 1:
        return "PASS"
    
    # Dada a ação, retorna a jogada no formato "X,Y"
    row = action // n_board
    col = action % n_board
    
    return f"MOVE {row},{col}"


# Função para extrair a jogada do agente adversário
def extract_row_col(move_str):
    row, col = map(int, move_str.split(' ')[1].split(','))
    return row, col

# Função para se conectar ao servidor
def connect_to_server(host='localhost', port=12345):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((host, port))
    
    response = client_socket.recv(1024).decode()
    print(f"Server ResponseINIT: {response}")
    
    Game = response[-4:]
    print("Playing:", Game)
    
    initial_board = np.zeros((n_board, n_board),dtype=int)# tabuleiro inicial
    Go = GameState(initial_board) # jogo iniciado
    
    # Verifica se o server lhe atribuiu o agente 1 ou 2
    if "1" in response:
        ag=1
    else:
        ag=2
    first=True
    
    # Inicia os estados do jogo
    state = game.get_initial_state()
    previous_state = state
    inv = 0 # Contador de jogadas inválidas
    
    while True:
        # Se é a vez do agente 1, gera a jogada e envia para o server
        if ag == 1 or not first:
            move = generate_move_from_model(state, previous_state, ag)
            time.sleep(1)
            client_socket.send(move.encode())
            print("Send:", move)

            # Espera pela resposta do server
            response = client_socket.recv(1024).decode()
            print(f"Server Response1: {response}")
            if "END" in response:
                break
            if "VALID" == response:
                if move == "PASS":
                    Go = Go.pass_turn()
                else:
                    row, col = extract_row_col(move)
                    Go = Go.move(row, col)
                print(Go.board)
            # Se a jogada for inválida, tenta enviar outra jogada
            elif "INVALID" in response:
                while "VALID" != response:
                    move = generate_move_from_model(state, previous_state, ag)
                    time.sleep(1)
                    client_socket.send(move.encode())
                    print("Send:", move)
                    response = client_socket.recv(1024).decode()
                    inv += 1
                    print(f"Server Response1: {response}")
                    # Se o contador de jogadas inválidas chegar a 3, o agente passa a vez
                    if inv == 3:
                        break
                # Se a jogada for válida, executa a jogada
                if "VALID" == response:
                    if move == "PASS":
                        Go = Go.pass_turn() # executa o pass_turn
                        print(f"Server Response1: {response}")
                    else:
                        row, col = extract_row_col(move)
                        Go = Go.move(row, col) # executa o move
                        print(f"Server Response1: {response}")
                    print(Go.board)
                if "TURN LOSS" == response:
                    Go = Go.pass_turn() # executa o pass_turn
                    print(Go.board)
                    state = Go.board # atualiza o estado do jogo
        first = False
        
        # Espera pela resposta do server, que é a jogada do agente adversário
        response = client_socket.recv(1024).decode()
        print(f"Server Response2: {response}")
        
        # Se o jogo acabar, sai do ciclo
        if "END" in response:
            break
        
        # Se o agente adversário passar a vez, executa o pass_turn
        if "PASS" in response:
            Go = Go.pass_turn()
            state = Go.board
            continue
        else:
            # Atualiza o estado do jogo dado a jogada do agente adversário
            row, col = extract_row_col(response)
            Go = Go.move(row, col)
            print(Go.board)
            # Atualiza o estado do jogo anterior
            previous_state = state

        state = Go.board
        if "END" in response:
            break

    # Fecha a conexão ao server
    client_socket.close()

if __name__ == "__main__":
    connect_to_server()